In [ ]:
import os
from pathlib import Path
import cv2
import numpy as np
from tqdm import tqdm
from IPython.display import display
import matplotlib.pyplot as plt

def unsharp_mask(image, blur_ksize=(5,5), amount=1.5, threshold=0):
    blurred = cv2.GaussianBlur(image, blur_ksize, 0)
    sharp = float(amount + 1) * image - float(amount) * blurred
    sharp = np.maximum(sharp, np.zeros(sharp.shape))
    sharp = np.minimum(sharp, 255 * np.ones(sharp.shape))
    sharp = sharp.round().astype(np.uint8)
    if threshold > 0:
        low_contrast_mask = np.absolute(image - blurred) < threshold
        np.copyto(sharp, image, where=low_contrast_mask)
    return sharp


def sharpen_image(image, strength=1.5):
    # Convert image to float32 for precision
    blurred = cv2.GaussianBlur(image, (0, 0), sigmaX=3)
    sharpened = cv2.addWeighted(image, 1 + strength, blurred, -strength, 0)
    return sharpened

def laplacian_sharpen(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    lap = cv2.Laplacian(gray, cv2.CV_64F)
    lap = np.uint8(np.clip(lap, 0, 255))
    sharpened = cv2.addWeighted(image, 1.0, cv2.cvtColor(lap, cv2.COLOR_GRAY2BGR), 0.3, 0)
    return sharpened

def compute_edge_density(gray_image):
    edges = cv2.Canny(gray_image, 25, 75)
    edge_pixels = np.sum(edges > 0)
    total_pixels = edges.size
    density = edge_pixels / total_pixels
    return density, edges

def enhance_dark_regions(image):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    dark_mask = cv2.inRange(gray, 0, 50)
    dark_highlight = cv2.bitwise_and(image, image, mask=dark_mask)
    image = cv2.addWeighted(image, 1.0, dark_highlight, 0.3, 0)
    return image

def apply_prewitt_filter(image):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    kernelx = np.array([[ -1, 0, 1], 
                        [ -1, 0, 1], 
                        [ -1, 0, 1]], dtype=np.float32)
    kernely = np.array([[  1,  1,  1], 
                        [  0,  0,  0], 
                        [ -1, -1, -1]], dtype=np.float32)

    grad_x = cv2.filter2D(gray, -1, kernelx)
    grad_y = cv2.filter2D(gray, -1, kernely)

    prewitt = cv2.addWeighted(grad_x, 0.5, grad_y, 0.5, 0)
    prewitt_colored = cv2.cvtColor(prewitt, cv2.COLOR_GRAY2RGB)

    return prewitt_colored

def preprocessImageWithDebug(
    imagePath,
    apply_white_balance=True,
    apply_clahe=True,
    apply_gamma=True,
    apply_edge_preserving=False,
    apply_median_blur=False,
    apply_gradient_map=False,
    apply_bilateral_filter=True,
    resize_dim=(640, 640),
    show_steps=True,
    apply_prewitt=True
):
    steps = []
    titles = []

    image = cv2.imread(imagePath)
    if image is None:
        return None
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    steps.append(image.copy())
    titles.append("Original")

    if apply_median_blur:
        image = cv2.medianBlur(image, 3)
        steps.append(image.copy())
        titles.append("Median blur Image")
        
    if apply_white_balance:
        wb = cv2.xphoto.createSimpleWB()
        image = wb.balanceWhite(image)
        steps.append(image.copy())
        titles.append("White Balanced")
    
    if apply_edge_preserving:
        image = cv2.edgePreservingFilter(image, flags=1, sigma_s=60, sigma_r=0.4)
        steps.append(image.copy())
        titles.append("Edge Preserving")
        
    enhanced_image = enhance_dark_regions(image)
    gray_dbg = cv2.cvtColor(enhanced_image, cv2.COLOR_RGB2GRAY)
    edge_density, edge_map = compute_edge_density(gray_dbg)
    steps.append(cv2.cvtColor(edge_map, cv2.COLOR_GRAY2RGB))
    titles.append(f"Edge Map (density={edge_density:.3f})")

    if edge_density < 0.005: # to prevent detecting black items as empty spaces
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        contrast_img = clahe.apply(gray_dbg)
        image = cv2.cvtColor(contrast_img, cv2.COLOR_GRAY2RGB)
        steps.append(image.copy())
        titles.append("CLAHE for Low-Edge Region")

    if apply_clahe:
        lab = cv2.cvtColor(image, cv2.COLOR_RGB2LAB)
        l, a, b = cv2.split(lab)
        clahe = cv2.createCLAHE(clipLimit=1.0, tileGridSize=(8, 8))
        l_clahe = clahe.apply(l)
        lab = cv2.merge((l_clahe, a, b))
        image = cv2.cvtColor(lab, cv2.COLOR_LAB2RGB)
        steps.append(image.copy())
        titles.append("CLAHE")

    if apply_gamma:
        gamma = 1.05
        inv_gamma = 1.0 / gamma
        table = np.array([(i / 255.0) ** inv_gamma * 255 for i in np.arange(256)]).astype("uint8")
        image = cv2.LUT(image, table)
        steps.append(image.copy())
        titles.append(f"Gamma {gamma}")    

    if apply_gradient_map:
        gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
        sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
        gradient = cv2.magnitude(sobelx, sobely)
        gradient = np.clip(gradient / gradient.max() * 255, 0, 255).astype(np.uint8)
        gradient_colored = cv2.cvtColor(gradient, cv2.COLOR_GRAY2RGB)
        image = cv2.addWeighted(image, 0.8, gradient_colored, 0.2, 0)
        steps.append(image.copy())
        titles.append("Gradient Overlay")
    
    if apply_prewitt:
        edge_overlay = apply_prewitt_filter(image)
        image = cv2.addWeighted(image, 0.8, edge_overlay, 0.2, 0)

    if apply_bilateral_filter:
        image = cv2.bilateralFilter(image, d=3, sigmaColor=15, sigmaSpace=15)
        steps.append(image.copy())
        titles.append("Bilateral Filter")

    image = cv2.resize(image, resize_dim, interpolation=cv2.INTER_LINEAR)
    
    image = sharpen_image(image, strength=2.0)
    steps.append(image.copy())
    titles.append("Sharpened Image")

    image = unsharp_mask(image, amount=2.0, threshold=10)
    steps.append(image.copy())
    titles.append("Unsharp mask Image")

    steps.append(image.copy())
    titles.append("Final Resized")
    if show_steps:
        fig, axs = plt.subplots(1, len(steps), figsize=(4 * len(steps), 5))
        if len(steps) == 1:
            axs = [axs]
        for i, (img, title) in enumerate(zip(steps, titles)):
                axs[i].imshow(img)
                axs[i].set_title(title, fontsize=10)
                axs[i].axis('off')
        
        plt.tight_layout()    
        display(fig)
            

    return cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

In [ ]:
def processAndSaveImages(inputRoot, outputRoot):
    """Process and save images for YOLOv5 format."""
    splits = ['train', 'test', 'valid'] 

    for split in splits:
        inputDir = os.path.join(inputRoot, split, 'images')
        outputDir = os.path.join(outputRoot, f"{split}", 'images')
        Path(outputDir).mkdir(parents=True, exist_ok=True)

        imageFiles = [f for f in os.listdir(inputDir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

        for imageFile in tqdm(imageFiles, desc=f"Processing {split} images"):
            inputPath = os.path.join(inputDir, imageFile)
            processedImg = preprocessImageWithDebug(inputPath, apply_edge_preserving=False, apply_median_blur=True, apply_gradient_map=False, apply_bilateral_filter=False, show_steps=False)
            outputPath = os.path.join(outputDir, imageFile)
            cv2.imwrite(outputPath, processedImg)

inputRoot = "oos/Original"
outputRoot = "oos"

processAndSaveImages(inputRoot, outputRoot)



In [ ]:
!python train.py --img 416 --batch 64 --epochs 10 --data oos/data.yaml --weights yolov5m.pt --name oos_preprocessed_detector

In [ ]:
!python val.py --weights runs/train/oos_preprocessed_detector11/weights/best.pt --data oos/data.yaml --img 640 --save-txt --save-conf

In [ ]:
!python detect.py --weights runs/train/oos_preprocessed_detector11/weights/best.pt --img 640 --conf 0.50 --source oos/test/images